<FONT size="5.9pt"> **NB2 - PROSPECTIVE CONSEQUENTIAL LCA OF BARK BIOREFINING - PART 1**
</FONT> 

This Jupyter Notebook was used to conduct an environmental prospective consequential LCA study comparing bark combustion and biorefinery processes (see main manuscript). The [*lca_algebraic*](https://link.springer.com/article/10.1007/s11367-021-01993-z) library was used to build the parameterised LCA model; refer to its [documentation](https://github.com/oie-mines-paristech/lca_algebraic/tree/master) for details. Time-differenciated climate impacts were calculated using dynamic characterisation factors (DCFs) from Levasseur et al. (2010) over a 100-year time horizon, based on the ALIGNED guidelines ([Hamelin et al., 2024](https://zenodo.org/records/10843259)). Other impact categories from the IPCC2021 and ReCiPe 2016 LCIA methods were also included. This Part 1 builds a parameterised model to perform the comparative LCA study using the IPCC 2021 and ReCiPe 2016 LCIA methods. The time-differentiated climate impacts are calculated in Part 2. 

# 1. Set-up the directories <a class="anchor" id="section1"></a> 

It is advised to create a new folder in which the Jupyter Notebooks are downloaded from the GitHub repository and saved. A "results" folder will be created to store all outputs of the code (e.g., graphs, tables). This code uploads the ecoinvent database from a local directory. Therefore, a license for the ecoinvent is required and the database should be stored on your computer directly.

In [ ]:
import os
from shutil import copyfile
import pathlib
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# define the initial working directory
wdir = Path().resolve() # get the directory of the notebook
pdir = os.path.join(wdir,'projects')
print(f"Working Directory: {wdir}")

# set the BW project directory
pathlib.Path(pdir).mkdir(parents=True, exist_ok=True)
os.environ['BRIGHTWAY2_DIR'] = pdir
print('Project directory:', os.environ['BRIGHTWAY2_DIR'])

# set the directory to store the results
result_dir = os.path.join(wdir,'results')   
pathlib.Path(result_dir).mkdir(parents=True, exist_ok=True)
print('Results directory:', result_dir)

# set the directory in which the databases are stored
db_dir = os.path.join(wdir,'databases')
pathlib.Path(db_dir).mkdir(parents=True, exist_ok=True)
print('Databases directory:', db_dir)

# 2. Import background databases <a class="anchor" id="section2"></a>

The background databases used in this study correspond to ecoinvent 3.9 consequential and biosphere3. Several versions of the ecoinvent database were generated using the [*premise*](https://www.sciencedirect.com/science/article/pii/S136403212200226X) Python library (see Jupyter Notebook NB1). 

In [ ]:
import lca_algebraic as alg
import brightway2 as bw

# initiate the project / choose project name
alg.initProject('lca_bark') ### NAME OF THE PROJECT CAN BE CHANGED

# import the ecoinvent database from a local directory
alg.importDb("ecoinvent_3.9_conseq", str(db_dir + '/ecoinvent 3.9_consequential_ecoSpold02/datasets')) ## VERSION OF ECOINVENT TO BE CHANGED

# overview of the databases
alg.list_databases()

# 3. Create foreground databases <a class="anchor" id="section3"></a>

In [ ]:
USER_DB = 'Foreground DB' ## NAME OF THE FOREGROUND DATABASE CAN BE CHANGED
alg.resetDb(USER_DB)
alg.resetParams()

## 3.1. Get ecoinvent datasets

In [ ]:
# Choose which version of ecoinvent to perform the LCA calculations with
#ei = 'ecoinvent_3.9_conseq'
#ei = 'ecoinvent_consequential_3.9_remind_SSP2-PkBudg500_2045'
#ei = 'ecoinvent_consequential_3.9_remind_SSP2-Base_2045'
#ei = 'ecoinvent_consequential_3.9_remind_SSP1-PkBudg500_2045'
ei = 'ecoinvent_consequential_3.9_image_SSP2-Base_2045'

In [ ]:
# technopshere flows
grid_SE = alg.findActivity(name='market for electricity, high voltage', loc = 'SE', db_name=ei) 
grid_RER = alg.findActivity(name='market group for electricity, high voltage', loc = 'RER', db_name=ei) 
ethylacetate = alg.findActivity(name='ethyl acetate production', loc = 'RER', db_name=ei) 
CO2 = alg.findActivity(name='carbon dioxide production, liquid', loc = 'RER', db_name=ei)
sodiumbicarbonate = alg.findActivity(name='market for sodium bicarbonate', loc = 'GLO', db_name=ei)
sodiumhydroxide = alg.findActivity(name='market for sodium hydroxide, without water, in 50% solution state', loc = 'GLO', db_name=ei)
quicklime = alg.findActivity(name='quicklime production, milled, loose', loc = 'CH', db_name=ei)
wwt = alg.findActivity(name='treatment of wastewater, average, wastewater treatment', loc = 'Europe without Switzerland', db_name=ei)
mw_RER = alg.findActivity(name='market group for municipal solid waste', loc = 'RER', db_name=ei) 
heatwoodchips = alg.findActivity(name='heat production, wood chips from industry, at furnace 50kW', loc = 'CH', db_name=ei)

# substituted products
talloil = alg.findActivity(name='tall oil refinery operation', loc = 'GLO', db_name=ei) 
potatostarch = alg.findActivity(name='potato starch production', loc = 'DE', db_name=ei) 
phenol = alg.findActivity(name='phenol production, from cumene', loc = 'RER', db_name=ei)
pulp = alg.findActivity(name='sulfate pulp production, from softwood, unbleached', loc = 'RER', db_name=ei)
limestone = alg.findActivity(name='market for limestone residue', loc = 'CH', db_name=ei)

# biosphere flows
ethylacetateemission = alg.findActivity(name='Ethyl acetate', db_name='biosphere3', categories = ('air',))
water = alg.findActivity(name='Water, river', db_name='biosphere3', categories = ('natural resource','in water'))
carbondioxide = alg.findActivity(name='Carbon dioxide, fossil', db_name='biosphere3', categories = ('air',))
carbondioxidenonfossil = alg.findActivity(name='Carbon dioxide, non-fossil', db_name='biosphere3', categories = ('air',))

## 3.2. Modify the ecoinvent dataset for heat production via cogeneration

The ecoinvent dataset for heat production from wood chips was used to model the environmental burdens of marginal heat supply in both biorefinery configurations. It was also used to calculate the impacts of bark combustion in the combustion scenario and residual biomass combustion in the biorefineries. The dataset was adapted for each specific application (see below).

In [ ]:
# dataset selected in ecoinvent
heat = alg.findActivity(name='heat and power co-generation, wood chips, 6667 kW, state-of-the-art 2014', loc = 'SE', db_name=ei)

### 3.2.1. Combustion of bark 

The new dataset "heat_CHP_bark" includes the impacts associated with the cogeneration process itself, excluding those from wood chip production (assumed similar to bark) and direct non-fossil emissions from wood combustion. The system boundaries exclude bark production (as this process is similar in all three assessed scenarios), while biogenic carbon emissions are accounted for separately. Since the properties of wood chips and bark are similar, it was assumed that combustion processes would result in the same impacts regardless of whether wood chips or bark are used as input.

In [ ]:
heat_CHP_bark = alg.copyActivity(USER_DB, heat, code = 'heat and power co-generation, wood chips, 6667 kW, state-of-the-art 2014', withExchanges=True)
heat_CHP_bark.deleteExchanges('Carbon dioxide, non-fossil')
heat_CHP_bark.deleteExchanges('Carbon monoxide, non-fossil')
if ei == 'ecoinvent_3.9_conseq':
    heat_CHP_bark.deleteExchanges('wood chips, wet, measured as dry mass') # for ecoinvent 3.9
else:    
    heat_CHP_bark.deleteExchanges('market for wood chips, wet, measured as dry mass')
#alg.printAct(heat_CHP_bark)

### 3.2.2. Burdens from marginal heat production

The unmodified ecoinvent dataset was used as a basis. Here, the objective is to model the heat production from wood chips that would be required to compensate for the heat produced when bark is combusted in the business-as-usual scenario (i.e. combustion scenario). The total amount of heat that would need to be produced for compensation corresponds to the total amount of heat resulting from the combustion of 1 kg DW-eq of bark minus the heat produced via combustion of the residual biomass obtained in the biorefinery. The original process extracted from ecoinvent is not changed since the impacts from wood chips production and combustion are accounted for. The biogenic CO2 and CO from the cogeneration process are not removed. The original activity is simply copied and given a new name for distinction.

In [ ]:
heat_CHP_woodchips = alg.copyActivity(USER_DB, heat, code = 'heat and power co-generation, wood chips, 6667 kW, state-of-the-art 2014', withExchanges=True)

### 3.2.3. Heat production from residual biomass

Again, the same ecoinvent dataset is used as a basis. The amount of residual biomass produced at each processing step is calculated from the mass and carbon balance. The amount of heat produced is calculated using the relation: 1 kg dry mass => 8.5 MJ according to the ecoinvent dataset for heat from cogeneration using wood chips. The same dataset as the one used for heat production from bark is used. No impacts from the biomass and no direct emissions are accounted for. A new name is given for distinction. 

In [ ]:
heat_CHP_RB = alg.copyActivity(USER_DB, heat, code = 'heat and power co-generation, wood chips, 6667 kW, state-of-the-art 2014', withExchanges=True)
heat_CHP_RB.deleteExchanges('Carbon dioxide, non-fossil')
heat_CHP_RB.deleteExchanges('Carbon monoxide, non-fossil')
if ei == 'ecoinvent_3.9_conseq':
    heat_CHP_RB.deleteExchanges('wood chips, wet, measured as dry mass') # ecoinvent 3.9
else:
    heat_CHP_RB.deleteExchanges('market for wood chips, wet, measured as dry mass')

In [ ]:
# check the climae impacts of heat production using IPCC 2021
ipcc2021 = alg.findMethods('global warming potential (GWP100)', 'IPCC 2021')[0]
climate_impacts_heat_CHP_RB = alg.compute_impacts([heat_CHP_RB],[ipcc2021])
climate_impacts_heat_CHP_RB

# 4. Create foreground parameters <a class="anchor" id="section4"></a>

## 4.1. Evaluate the influence of the geographical location

In the sensititvity analysis, the influence of the geographical location on the LCA results is analysed. There are no European market for heat production from cogeneration in the consequential ecoinvent v.3.9 database. However, the dataset exists for many locations. The only differences between the datasets are the electricity mix and the municipal solid waste treatment. Here, both the electricity mix and the waste treatment processes are changed based on the geographical location. List of countries analysed in the sensitivity analysis: SE, NO, FI, DK, FR, DE, BE, LU, NL, AT.

In [ ]:
# SE and RER have already been created above
grid_NO = alg.findActivity(name='market for electricity, high voltage', loc = 'NO', db_name=ei) 
grid_FI = alg.findActivity(name='market for electricity, high voltage', loc = 'FI', db_name=ei) 
grid_DK = alg.findActivity(name='market for electricity, high voltage', loc = 'DK', db_name=ei) 
grid_FR = alg.findActivity(name='market for electricity, high voltage', loc = 'FR', db_name=ei) 
grid_DE = alg.findActivity(name='market for electricity, high voltage', loc = 'DE', db_name=ei) 
grid_BE = alg.findActivity(name='market for electricity, high voltage', loc = 'BE', db_name=ei) 
grid_LU = alg.findActivity(name='market for electricity, high voltage', loc = 'LU', db_name=ei) 
grid_NL = alg.findActivity(name='market for electricity, high voltage', loc = 'NL', db_name=ei) 
grid_AT = alg.findActivity(name='market for electricity, high voltage', loc = 'AT', db_name=ei) 

# RER has already been created above
mw_SE = alg.findActivity(name='market for municipal solid waste', loc = 'SE', db_name=ei) 
mw_NO = alg.findActivity(name='market for municipal solid waste', loc = 'NO', db_name=ei) 
mw_FI = alg.findActivity(name='market for municipal solid waste', loc = 'FI', db_name=ei) 
mw_DK = alg.findActivity(name='market for municipal solid waste', loc = 'DK', db_name=ei) 
mw_FR = alg.findActivity(name='market for municipal solid waste', loc = 'FR', db_name=ei) 
mw_DE = alg.findActivity(name='market for municipal solid waste', loc = 'DE', db_name=ei) 
mw_BE = alg.findActivity(name='market for municipal solid waste', loc = 'BE', db_name=ei) 
mw_LU = alg.findActivity(name='market for municipal solid waste', loc = 'LU', db_name=ei) 
mw_NL = alg.findActivity(name='market for municipal solid waste', loc = 'NL', db_name=ei) 
mw_AT = alg.findActivity(name='market for municipal solid waste', loc = 'AT', db_name=ei) 

In [ ]:
# create switch parameter for electricity mix - default value is SE
grid_loc = alg.newEnumParam(
    name = 'grid_loc',
    values = ['grid_SE','grid_NO','grid_FI','grid_DK','grid_FR','grid_DE','grid_BE','grid_LU','grid_NL','grid_AT','grid_RER'],
    default = 'grid_SE',
    #label = 'source of electricity - biorefinery',
    description = "Switch on the source of electricty used in the biorefinery",
    group = 'energy')

# create the virtual activity corresponding to the switch
grid_switch_act = alg.newSwitchAct(dbname = USER_DB,
                                   name = 'grid_switch_act',
                                   paramDef = grid_loc,
                                   acts_dict = {'grid_SE' : grid_SE,
                                                'grid_NO': grid_NO,
                                                'grid_FI': grid_FI,
                                                'grid_DK': grid_DK,
                                                'grid_FR': grid_FR,
                                                'grid_DE': grid_DE,
                                                'grid_BE': grid_BE,
                                                'grid_LU': grid_LU,
                                                'grid_NL': grid_NL,
                                                'grid_AT': grid_AT,
                                                'grid_RER': grid_RER})

In [ ]:
# create switch parameter for municipal solid waste treatment - default value is RER
mw_loc = alg.newEnumParam(
    name = 'mw_loc',
    values = ['mw_SE','mw_NO','mw_FI','mw_DK','mw_FR','mw_DE','mw_BE','mw_LU','mw_NL','mw_AT','mw_RER'],
    default = 'mw_RER',
    #label = 'source of electricity - biorefinery',
    description = "Switch on the source of electricty used in the biorefinery",
    group = 'waste')

# create the virtual activity corresponding to the switch
mw_switch_act = alg.newSwitchAct(dbname = USER_DB,
                                   name = 'mw_switch_act',
                                   paramDef = mw_loc,
                                   acts_dict = {'mw_SE': mw_SE,
                                                'mw_NO': mw_NO,
                                                'mw_FI': mw_FI,
                                                'mw_DK': mw_DK,
                                                'mw_FR': mw_FR,
                                                'mw_DE': mw_DE,
                                                'mw_BE': mw_BE,
                                                'mw_LU': mw_LU,
                                                
                                                'mw_NL': mw_NL,
                                                'mw_AT': mw_AT,
                                                'mw_RER': mw_RER})

## 4.2. Evaluate the influence of the percentage of losses 

The float parameter defined corresponds to the percentage of biomass losses in the biorefinery processes. An increase in the percentage of losses means that less of the (co)products are produced but more heat is generated from the combustion of residual biomass.

In [ ]:
percentage_losses = alg.newFloatParam(name = "percentage_losses",
                           default = 0, min = 0, max = 1, 
                           unit = '%',
                           group = 'biomass')

## 4.3. Evaluate the influence of the integration of the biorefinery with co-located source of energy

The boolean parameter defined changes the amount of electricity used in the biorefinery processes. In the integrated biorefinery (standalone = 0), no external electricity is required and all the energy is supplied from the co-located source which comes into the system free of burdens. In the standalone biorefinery (standalone = 1), the electricity required is supplied by the grid. 

In [ ]:
standalone = alg.newBoolParam('standalone', 
                               default = 1, # 1 means the biorefinery system is stand-alone, 0 it is integrated
                               group = 'booleans')

# 5. Add the parameters to the foreground activities <a class="anchor" id="section5"></a>

## 5.1. Create parameterised heat production processes

The amount of electricity and municipal solid wastes are not changed, only the geographical location of the datasets. 

In [ ]:
heat_CHP_bark.updateExchanges({'electricity*':grid_switch_act})
heat_CHP_bark.updateExchanges({'municipal*':mw_switch_act})

heat_CHP_woodchips.updateExchanges({'electricity*':grid_switch_act})
heat_CHP_woodchips.updateExchanges({'municipal*':mw_switch_act})

heat_CHP_RB.updateExchanges({'electricity*':grid_switch_act})
heat_CHP_RB.updateExchanges({'municipal*':mw_switch_act})

## 5.2. Create parameterised biorefinery processes

All the activities are scaled to the functional unit i.e. the processing of 1 kg of bark dry mass. 

In [ ]:
# lipophilic extraction from bark
lipophilic_extraction = alg.newActivity(db_name = USER_DB, 
         name = 'lipophilic extraction',
         unit = 'unit',
         exchanges={
             # technosphere inputs
             ethylacetate: 0.002,
             grid_SE: 2.44*standalone,
             # emissions
             ethylacetateemission: 0.002
         })

lipophilic_extraction.updateExchanges({'electricity*':grid_switch_act})
#alg.printAct(lipophilic_extraction).to_excel(str(result_dir+'/lipophilic_extraction_'+ei+'.xlsx'))
alg.printAct(lipophilic_extraction)

In [ ]:
# sugar extraction from lipophilic depleted bark
sugar_extraction = alg.newActivity(db_name = USER_DB, 
 name = 'sugar extraction',
 unit = 'unit',
 exchanges={
     grid_SE: 5*standalone,
     wwt: -0.00007,
     # natural resources
     water: 0.00007
 })

sugar_extraction.updateExchanges({'electricity*':grid_switch_act})
#alg.printAct(sugar_extraction).to_excel(str(result_dir+'/sugar_extraction_'+ei+'.xlsx'))
alg.printAct(sugar_extraction)

In [ ]:
sugar_extraction_withlosses = alg.newActivity(db_name = USER_DB,
                                     name = 'sugar_extraction_withlosses',
                                     unit = 'unit',
                                     exchanges={sugar_extraction:1 # - percentage_losses
                                    })

In [ ]:
# pulping of sugar depleted bark
pulping = alg.newActivity(db_name = USER_DB, 
 name = 'pulping',
 unit = 'unit',
 exchanges={
     sodiumhydroxide: 0.00523,
     quicklime: 0.00629, # calcium oxide
     wwt: -0.00035,
     grid_SE: 1.03*standalone,
     heatwoodchips:0.35,
     CO2: 0.124*standalone,
     limestone:-0.00551,
     # natural resources
     water: 0.00035
 })

pulping.updateExchanges({'electricity*':grid_switch_act})
#alg.printAct(pulping).to_excel(str(result_dir+'/pulping_'+ei+'.xlsx'))
alg.printAct(pulping)

In [ ]:
pulping_withlosses = alg.newActivity(db_name = USER_DB,
                                     name = 'pulping_withlosses',
                                     unit = 'unit',
                                     exchanges={pulping:1 #- percentage_losses
                                               })

In [ ]:
# production of heat from residual biomass via cogeneration
heat_residues = alg.newActivity(db_name = USER_DB, 
 name = 'heat (RB)',
 unit = 'unit',
 exchanges={
     heat_CHP_RB: 1.53*(1+percentage_losses)
 })
alg.printAct(heat_residues)

In [ ]:
# conversion of lipophilic extractives into tall oil (no impacts assumed)
processA = alg.newActivity(db_name = USER_DB, 
 name = 'process A',
 unit = 'unit',
 exchanges={
 })

In [ ]:
# conversion of sugar extractives into starch (no impacts assumed)
processB = alg.newActivity(db_name = USER_DB, 
 name = 'process B',
 unit = 'unit',
 exchanges={
 })

In [ ]:
# conversion of lignin into phenol (no impacts assumed)
processC = alg.newActivity(db_name = USER_DB, 
 name = 'process C',
 unit = 'unit',
 exchanges={
 })

## 5.3. Create parameterised end-of-life processes

The end-of-life (EoL) processes were differentiated for each product and application since the lifespan of the products varies (and will therefore influence the calculation of the climate impacts using dynamic chatacterisation factors). All products are considered to be solid wastes treated by incineration at their EoL. The amount of waste treated depends on the percentage of biomass losses in the biorefinery processes. More losses means that less products are generated and therefore less wastes are treated. 

| Product            | Application          | Share   | Lifespan       | Amount   | Unit |
|--------------------|----------------------|---------|----------------|----------|------|
| Pulp               | Biomedical products | 33.33%  | <1 year        | 0.1367   | kg   |
| Pulp               | Textiles            | 33.33%  | 5 years        | 0.1367   | kg   |
| Pulp               | Nanocomposites      | 33.33%  | 20 years       | 0.1367   | kg   |
| Phenol             | Automotive          | 40%     | 13 years       | 0.068    | kg   |
| Phenol             | Construction        | 23%     | 50 years       | 0.0391   | kg   |
| Phenol             | Furniture           | 18%     | 15 years       | 0.0306   | kg   |
| Phenol             | Electronics         | 14%     | 7 years        | 0.0238   | kg   |
| Phenol             | Others              | 5%      | 21.25 years    | 0.0085   | kg   |
| Starch             | Cardboard boxes     | 100%    | 1 year         | 0.22     | kg   |
| Tall oil           | Biofuel             | 100%    | <1 year        | 0.02     | kg   |
| Residual biomass   | Heat production     | 100%    | <1 year        | 1.52915  | MJ   |


In [ ]:
# treatment of the biorefinery products at their EoL (aggregated value)
# amount corresponds to the mass of the biorefinery products excluding the residues (which are combusted for heat production)
EoL = alg.newActivity(db_name = USER_DB, 
 name = 'EoL',
 unit = 'unit',
 exchanges={
     #mw_RER: -(0.82*(1-percentage_losses))
     mw_RER: -(1-0.18*(1+percentage_losses))
 })
EoL.updateExchanges({'municipal*':mw_switch_act})
#alg.printAct(EoL).to_excel(str(result_dir+'/EoL_'+ei+'.xlsx'))
alg.printAct(EoL)

In [ ]:
# treatment of the pulp for biomedical application
EoL_pulp_biomed = alg.newActivity(db_name = USER_DB,
                                  name = 'EoL_pulp_biomed',
                                  unit = 'unit',
                                  exchanges={
                                      #mw_RER: -(0.3333*0.41*(1-percentage_losses))
                                      mw_RER: -(0.3333*(0.41/0.82)*(1-0.18*(1+percentage_losses)))
                                  })
EoL_pulp_biomed.updateExchanges({'municipal*':mw_switch_act})
#alg.printAct(EoL_pulp_biomed).to_excel(str(result_dir+'/EoL_pulp_biomed_'+ei+'.xlsx'))
alg.printAct(EoL_pulp_biomed)

In [ ]:
# treatment of the pulp for textile application
EoL_pulp_textile = alg.newActivity(db_name = USER_DB,
                                  name = 'EoL_pulp_textile',
                                  unit = 'unit',
                                  exchanges={
                                      #mw_RER: -(0.3333*0.41*(1-percentage_losses))
                                      mw_RER: -(0.3333*(0.41/0.82)*(1-0.18*(1+percentage_losses)))
                                  })
EoL_pulp_textile.updateExchanges({'municipal*':mw_switch_act})
#alg.printAct(EoL_pulp_textile).to_excel(str(result_dir+'/EoL_pulp_textile_'+ei+'.xlsx'))
alg.printAct(EoL_pulp_textile)

In [ ]:
# treatment of the pulp for nanocomposite application
EoL_pulp_nanocomposite = alg.newActivity(db_name = USER_DB,
                                  name = 'EoL_pulp_nanocomposite',
                                  unit = 'unit',
                                  exchanges={
                                      #mw_RER: -(0.3333*0.41*(1-percentage_losses))
                                      mw_RER: -(0.3333*(0.41/0.82)*(1-0.18*(1+percentage_losses)))
                                  })
EoL_pulp_nanocomposite.updateExchanges({'municipal*':mw_switch_act})
#alg.printAct(EoL_pulp_nanocomposite).to_excel(str(result_dir+'/EoL_pulp_nanocomposite_'+ei+'.xlsx'))
alg.printAct(EoL_pulp_nanocomposite)

In [ ]:
# treatment of the phenol for automotive application
EoL_phenol_auto = alg.newActivity(db_name = USER_DB,
                                  name = 'EoL_phenol_auto',
                                  unit = 'unit',
                                  exchanges={
                                      #mw_RER: -(0.4*0.17*(1-percentage_losses))
                                      mw_RER: -(0.4*(0.17/0.82)*(1-0.18*(1+percentage_losses)))
                                  })
EoL_phenol_auto.updateExchanges({'municipal*':mw_switch_act})
#alg.printAct(EoL_phenol_auto).to_excel(str(result_dir+'/EoL_phenol_auto_'+ei+'.xlsx'))
alg.printAct(EoL_phenol_auto)

In [ ]:
# treatment of the phenol for construction application
EoL_phenol_construction = alg.newActivity(db_name = USER_DB,
                                  name = 'EoL_phenol_construction',
                                  unit = 'unit',
                                  exchanges={
                                      #mw_RER: -(0.23*0.17*(1-percentage_losses))
                                      mw_RER: -(0.23*(0.17/0.82)*(1-0.18*(1+percentage_losses)))
                                  })
EoL_phenol_construction.updateExchanges({'municipal*':mw_switch_act})
#alg.printAct(EoL_phenol_construction).to_excel(str(result_dir+'/EoL_phenol_construction_'+ei+'.xlsx'))
alg.printAct(EoL_phenol_construction)

In [ ]:
# treatment of the phenol for furniture application
EoL_phenol_furniture = alg.newActivity(db_name = USER_DB,
                                  name = 'EoL_phenol_furniture',
                                  unit = 'unit',
                                  exchanges={
                                      #mw_RER: -(0.18*0.17*(1-percentage_losses))
                                      mw_RER: -(0.18*(0.17/0.82)*(1-0.18*(1+percentage_losses)))
                                  })
EoL_phenol_furniture.updateExchanges({'municipal*':mw_switch_act})
#alg.printAct(EoL_phenol_furniture).to_excel(str(result_dir+'/EoL_phenol_furniture_'+ei+'.xlsx'))
alg.printAct(EoL_phenol_furniture)

In [ ]:
# treatment of the phenol for electronics application
EoL_phenol_electronics = alg.newActivity(db_name = USER_DB,
                                  name = 'EoL_phenol_electronics',
                                  unit = 'unit',
                                  exchanges={
                                      #mw_RER: -(0.14*0.17*(1-percentage_losses))
                                      mw_RER: -(0.14*(0.17/0.82)*(1-0.18*(1+percentage_losses)))
                                  })
EoL_phenol_electronics.updateExchanges({'municipal*':mw_switch_act})
#alg.printAct(EoL_phenol_electronics).to_excel(str(result_dir+'/EoL_phenol_electronics_'+ei+'.xlsx'))
alg.printAct(EoL_phenol_electronics)

In [ ]:
# treatment of the phenol for other applications
EoL_phenol_others = alg.newActivity(db_name = USER_DB,
                                  name = 'EoL_phenol_others',
                                  unit = 'unit',
                                  exchanges={
                                      #mw_RER: -(0.05*0.17*(1-percentage_losses))
                                      mw_RER: -(0.05*(0.17/0.82)*(1-0.18*(1+percentage_losses)))
                                  })
EoL_phenol_others.updateExchanges({'municipal*':mw_switch_act})
#alg.printAct(EoL_phenol_others).to_excel(str(result_dir+'/EoL_phenol_others_'+ei+'.xlsx'))
alg.printAct(EoL_phenol_others)

In [ ]:
# treatment of the phenol for other applications
EoL_starch_cardboard = alg.newActivity(db_name = USER_DB,
                                  name = 'EoL_starch_cardboard',
                                  unit = 'unit',
                                  exchanges={
                                      #mw_RER: -(0.22*(1-percentage_losses))
                                      mw_RER: -((0.22/0.82)*(1-0.18*(1+percentage_losses)))
                                  })
EoL_starch_cardboard.updateExchanges({'municipal*':mw_switch_act})
#alg.printAct(EoL_starch_cardboard).to_excel(str(result_dir+'/EoL_starch_cardboard_'+ei+'.xlsx'))
alg.printAct(EoL_starch_cardboard)

In [ ]:
# treatment of the phenol for other applications
EoL_talloil_biofuel = alg.newActivity(db_name = USER_DB,
                                  name = 'EoL_talloil_biofuel',
                                  unit = 'unit',
                                  exchanges={
                                      #mw_RER: -(0.02*(1-percentage_losses))
                                      mw_RER: -((0.02/0.82)*(1-0.18*(1+percentage_losses)))
                                  })
EoL_talloil_biofuel.updateExchanges({'municipal*':mw_switch_act})
#alg.printAct(EoL_talloil_biofuel).to_excel(str(result_dir+'/EoL_talloil_biofuel_'+ei+'.xlsx'))
alg.printAct(EoL_talloil_biofuel)

## 5.4. Create parameterised substituted processes

In [ ]:
# substitution of tall oil from pine wood (1:1)
talloil_sub = alg.newActivity(db_name = USER_DB,
                             name = 'tall oil (sub.)',
                             unit = 'unit',
                             exchanges = {talloil: - ((0.02/0.82)*(1-0.18*(1+percentage_losses)))})
alg.printAct(talloil_sub)

In [ ]:
# substitution of starch from potato (1:1)
starch_sub = alg.newActivity(db_name = USER_DB,
                          name = 'starch (sub.)',
                          unit = 'unit',
                          exchanges = {potatostarch: -((0.22/0.82)*(1-0.18*(1+percentage_losses)))})
alg.printAct(starch_sub)

In [ ]:
# substitution of phenol from cumene (1:1)
phenol_sub = alg.newActivity(db_name = USER_DB,
                            name = 'phenol (sub.)',
                            unit = 'unit',
                            exchanges = {phenol: -((0.17/0.82)*(1-0.18*(1+percentage_losses)))})
alg.printAct(phenol_sub)

In [ ]:
# substitution of sulfate pulp (1:1)
pulp_sub = alg.newActivity(db_name = USER_DB,
                            name = 'pulp (sub.)',
                            unit = 'unit',
                            exchanges = {pulp: -((0.41/0.82)*(1-0.18*(1+percentage_losses)))})
alg.printAct(pulp_sub)

In [ ]:
# environmental impacts from the production of marginal heat in the biorefinery scenarios (e.g., heat that would have been produced by directly burning the bark)
# amount corresponds to the total amount of heat produced when burning bark (8.5 MJ) minus the heat produced from the residues
heat_burdens = alg.newActivity(db_name = USER_DB, 
 name = 'heat (burd.)',
 unit = 'unit',
 exchanges={
     heat_CHP_woodchips: 8.5 -(1.53*(1+percentage_losses))
 })
alg.printAct(heat_burdens)

## 5.5. Scaled process for combustion scenario

In [ ]:
heat_CHP_bark_scaled = alg.newActivity(db_name = USER_DB,
                                     name = 'heat CHP bark',
                                     unit = 'unit',
                                     exchanges={heat_CHP_bark:8.5})

# 6. Calculate the static climate impacts using IPCC 2021 GWP100

## 6.1. Static climate impacts from indirect GHG emissions

In [ ]:
# static climate impacts (indirect emissions only) for the stand-alone biorefinery scenario
df_bioref_standalone = alg.compute_impacts([lipophilic_extraction, 
                                            sugar_extraction_withlosses, 
                                            pulping_withlosses, 
                                            heat_burdens, 
                                            #processA, 
                                            #processB, 
                                            #processC, 
                                            heat_residues, 
                                            # EoL
                                            EoL_pulp_biomed,
                                            EoL_pulp_textile, 
                                            EoL_pulp_nanocomposite,
                                            EoL_phenol_auto, 
                                            EoL_phenol_construction, 
                                            EoL_phenol_furniture, 
                                            EoL_phenol_electronics, 
                                            EoL_phenol_others, 
                                            EoL_starch_cardboard, 
                                            EoL_talloil_biofuel,                                   
                                            # substitution
                                            talloil_sub,
                                            starch_sub,
                                            phenol_sub, 
                                            pulp_sub],
                                            # lcia method
                                            [ipcc2021],
                                            # parameters
                                            standalone = 1,
                                            percentage_losses = 0,
                                            grid_loc = 'grid_SE',
                                            mw_loc = 'mw_RER'
                                          )
#df_bioref_standalone.to_excel(str(result_dir+"/climate_impacts_IPCC2021_static_standalone_bioref_" + str(ei)+ ".xlsx"))

In [ ]:
# static climate impacts (indirect emissions only) for the integrated biorefinery scenario
df_bioref_integrated = alg.compute_impacts([lipophilic_extraction, 
                                            sugar_extraction_withlosses, 
                                            pulping_withlosses, 
                                            heat_burdens, 
                                            #processA, 
                                            #processB, 
                                            #processC, 
                                            heat_residues, 
                                            # EoL
                                            EoL_pulp_biomed,
                                            EoL_pulp_textile, 
                                            EoL_pulp_nanocomposite,
                                            EoL_phenol_auto, 
                                            EoL_phenol_construction, 
                                            EoL_phenol_furniture, 
                                            EoL_phenol_electronics, 
                                            EoL_phenol_others, 
                                            EoL_starch_cardboard, 
                                            EoL_talloil_biofuel,                                   
                                            # substitution
                                            talloil_sub,
                                            starch_sub,
                                            phenol_sub, 
                                            pulp_sub],
                                            # lcia method
                                            [ipcc2021],
                                            # parameters
                                            standalone = 0,
                                            percentage_losses = 0,
                                            grid_loc = 'grid_SE',
                                            mw_loc = 'mw_RER'
                                          )
#df_bioref_integrated.to_excel(str(result_dir+"/climate_impacts_IPCC2021_static_integrated_bioref_" + str(ei)+ ".xlsx"))

In [ ]:
# static climate impacts (indirect emissions only) for the combustion scenario
df_combustion = alg.compute_impacts([heat_CHP_bark_scaled],
                            [ipcc2021],
                            # parameters
                            percentage_losses = 0,
                            grid_loc = 'grid_SE',
                            mw_loc = 'mw_RER'
                            )
#df_combustion.to_excel(str(result_dir+"/climate_impacts_IPCC2021_static_combustion_" + str(ei)+ ".xlsx"))

In [ ]:
# merge the three dataframes
merged_df = pd.concat([df_bioref_standalone, df_bioref_integrated, df_combustion], axis = 1)
# change the name of the columns
merged_df.columns = ['Stand-alone biorefinery', 'Integrated biorefinery', 'Combustion']
merged_df

When using the ecoinvent consequential v.9 database, the values above match the ones in the excel file sheet "GWP100AB_ei39" in which the values were calcualted using the Activity Browser and the IPCC2021 GWP100. The values are also almost similar to the the total GWP100 values calculated in the spreadhseet "GWP100total_ei39" even if the later includes CO2, CH4, and N2O emissions only (while the table above includes all GHGs). 

## 6.2. Static climate impacts from direct CO2 emissions

The calculation of the direct CO2-eq emissions is based on the carbon balance of the biorefinery (see Supplementary Material 1). Carbon capture is excluded here since the LCA study is limited to gate-to-gate system boundaries (see main manuscript). 

In [ ]:
carbon_content_df = pd.read_excel(str(str(wdir) + '/carbon_content.xlsx'), index_col=0)
carbon_content_df

In [ ]:
CO2_emissions_df = carbon_content_df.copy()
CO2_emissions_df['Stand-alone biorefinery'] = CO2_emissions_df['Stand-alone biorefinery']/1000*3.67
CO2_emissions_df['Integrated biorefinery'] = CO2_emissions_df['Integrated biorefinery']/1000*3.67
CO2_emissions_df['Combustion'] = CO2_emissions_df['Combustion']/1000*3.67
CO2_emissions_df = CO2_emissions_df.drop(['Year'], axis=1)
CO2_emissions_df.loc['Total']= CO2_emissions_df.sum()
CO2_emissions_df

## 6.3. Total static climate impacts

In [ ]:
total_df = merged_df.copy() 
total_df.loc['EoL']= total_df.iloc[5:15].sum()
total_df_vf = total_df.drop(total_df.index[5:15])

row_to_move1 = total_df_vf.iloc[[10]]
total_df_vf = total_df_vf.drop(['EoL'])
total_df_vf = pd.concat([total_df_vf.iloc[:5], row_to_move1, total_df_vf.iloc[5:]])

row_to_move2 = total_df_vf.iloc[[4]]
total_df_vf = total_df_vf.drop(['heat (RB)'])
total_df_vf = pd.concat([total_df_vf.iloc[:3], row_to_move2, total_df_vf.iloc[3:]])

## add a row from the direct CO2 emissions (biogenic)
total_df_vf.loc['direct CO2'] = CO2_emissions_df.loc['Total']

#change index labels
index_labels = ['lipophilic extraction', 'sugar extraction', 'pulping', 'heat (RB)', 'heat (burd.)','EoL', 
                'tall oil (sub.)', 'starch (sub.)', 'phenol (sub.)', 'pulp (sub.)', 'heat CHP bark', 'biogenic CO2']
total_df_vf.index = index_labels
total_df_vf.rename(columns={'Stand-alone biorefinery': 'Stand-alone\nbiorefinery', 'Integrated biorefinery': 'Integrated\nbiorefinery'}, inplace=True)
#total_df_vf.loc['Total']= total_df_vf.sum()
total_df_vf.to_excel(str(result_dir+'/total_static_climate_impacts_IPCC' + ei +'.xlsx')) 
total_df_vf

In [ ]:
# color palette from: https://davidmathlogic.com/colorblind/#%23332288-%23117733-%2344AA99-%2388CCEE-%23DDCC77-%23CC6677-%23AA4499-%23882255
colors = ['#DDCC77', # LE yellow
          '#88CCEE', # SE sky blue
          '#332288', # P dark blue
          '#fff0db', # heat (RB) beige
          'black', # heat burd. black
          '#882255',# EoL red wine
          '#D3D3D3', #tall oil sub.light grey
          '#44AA99', # starch sub. teal
          '#117733', # phenol sub. dark green 
          '#AA4499', # pulp sub.plum pink
          'grey', # heat CHP bark dark grey
          '#CC6677', # biogenic CO2 orange pink          
]

In [ ]:
# GWP scores with substitution
GWP_scores_ei39 = list(total_df_vf.sum())
GWP_scores_ei39 = ['%.2f' % elem for elem in GWP_scores_ei39]
GWP_scores_ei39 = [float(elem) for elem in GWP_scores_ei39] # the list items must be floats
GWP_scores_ei39

# GWP scores without substitution
df_wo_subst_ei39 = total_df_vf.drop(['heat (burd.)', 'tall oil (sub.)', 'starch (sub.)', 'phenol (sub.)', 'pulp (sub.)'])
GWP_scores_wo_subst_ei39 = list(df_wo_subst_ei39.sum())
GWP_scores_wo_subst_ei39 = ['%.2f' % elem for elem in GWP_scores_wo_subst_ei39]
GWP_scores_wo_subst_ei39 = [float(elem) for elem in GWP_scores_wo_subst_ei39] # the list items must be floats
GWP_scores_wo_subst_ei39

# transpose the dataframe
df_transposed = total_df_vf.T

fig, ax = plt.subplots(figsize=(7, 7))

ax.set_ylim(-0.7,4.0)

col_names = ['Stand-alone\nbiorefinery', 'Integrated\nbiorefinery', 'Combustion']

bars = df_transposed.plot(kind='bar', stacked=True, ax=ax, color=colors,legend = False, width=0.5)
ax.set_ylabel('Static climate impacts (kg CO2-eq/FU)', fontsize=14)
#ax.legend(loc='lower center', bbox_to_anchor=(0.65, 0.7), title_fontsize='12', fontsize='14', ncol=2, frameon = False)

ax.tick_params(axis='x', labelsize=14, rotation=0)
ax.tick_params(axis='y', labelsize=14)
ax.tick_params(axis='x', which='both', bottom=False)  # Remove x-axis ticks

ax.axhline(y=0, color='black', linestyle='-', linewidth=0.5)

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(False)

# add the single GWP100 scores for each scenario
categories_index = np.arange(len(col_names))
ax.scatter(categories_index, GWP_scores_wo_subst_ei39, color='black', marker = '^', zorder=5)
ax.scatter(categories_index, GWP_scores_ei39, color='black', marker = 'H', zorder=5)

for i, val in enumerate(GWP_scores_ei39):
    ax.text(categories_index[i], val - 0.12, f'{val}', ha='center', color='black',va = 'top', fontsize=14)

for i, val in enumerate(GWP_scores_wo_subst_ei39):
    ax.text(categories_index[i], val+0.12, f'{val}', ha='center', color='black',va = 'bottom', fontsize=14)

# +/- 0.08

plt.tight_layout()

#plt.savefig(str(result_dir+'/fig5_DCFs_results_' + ei + '.pdf'), dpi=300)
plt.savefig(str(result_dir+'/SM1_static_GWP100_results_'+ei+'.pdf'), dpi=300)

plt.show()

# 7. Calculate the impacts for other impact categories using ReCiPe

In [ ]:
recipe_all = alg.findMethods('ReCiPe 2016 v1.03, midpoint (H)')
recipe = recipe_all[18:36] # remove all the "no LT" impact categories
recipe_noGWP = [ic for ic in recipe if 'climate change' not in ic]
len(recipe_noGWP)

In [ ]:
df_bioref_standalone_allICs = alg.compute_impacts([lipophilic_extraction, 
                                            sugar_extraction_withlosses, 
                                            pulping_withlosses, 
                                            heat_burdens, 
                                            #processA, 
                                            #processB, 
                                            #processC, 
                                            heat_residues, 
                                            # EoL
                                            EoL_pulp_biomed,
                                            EoL_pulp_textile, 
                                            EoL_pulp_nanocomposite,
                                            EoL_phenol_auto, 
                                            EoL_phenol_construction, 
                                            EoL_phenol_furniture, 
                                            EoL_phenol_electronics, 
                                            EoL_phenol_others, 
                                            EoL_starch_cardboard, 
                                            EoL_talloil_biofuel,                                   
                                            # substitution
                                            talloil_sub,
                                            starch_sub,
                                            phenol_sub, 
                                            pulp_sub],
                                            # lcia method
                                            recipe_noGWP,
                                            # parameters
                                            standalone = 1,
                                            percentage_losses = 0,
                                            grid_loc = 'grid_SE',
                                            mw_loc = 'mw_RER'
                                          )
#df_bioref_standalone_allICs.to_excel(str(result_dir+"/all_impacts_recipe_standalone_bioref_" + str(ei)+ ".xlsx"))

In [ ]:
df_bioref_integrated_allICs = alg.compute_impacts([lipophilic_extraction, 
                                            sugar_extraction_withlosses, 
                                            pulping_withlosses, 
                                            heat_burdens, 
                                            #processA, 
                                            #processB, 
                                            #processC, 
                                            heat_residues, 
                                            # EoL
                                            EoL_pulp_biomed,
                                            EoL_pulp_textile, 
                                            EoL_pulp_nanocomposite,
                                            EoL_phenol_auto, 
                                            EoL_phenol_construction, 
                                            EoL_phenol_furniture, 
                                            EoL_phenol_electronics, 
                                            EoL_phenol_others, 
                                            EoL_starch_cardboard, 
                                            EoL_talloil_biofuel,                                   
                                            # substitution
                                            talloil_sub,
                                            starch_sub,
                                            phenol_sub, 
                                            pulp_sub],
                                            # lcia method
                                            recipe_noGWP,
                                            # parameters
                                            standalone = 0,
                                            percentage_losses = 0,
                                            grid_loc = 'grid_SE',
                                            mw_loc = 'mw_RER'
                                          )
#df_bioref_integrated_allICs.to_excel(str(result_dir+"/all_impacts_recipe_integrated_bioref_" + str(ei)+ ".xlsx"))

In [ ]:
df_combustion_allICs = alg.compute_impacts([heat_CHP_bark_scaled],
                            recipe_noGWP,
                            # parameters
                            percentage_losses = 0,
                            grid_loc = 'grid_SE',
                            mw_loc = 'mw_RER'
                            )
#df_combustion_allICs.to_excel(str(result_dir+"/all_impacts_recipe_integrated_combustion_" + str(ei)+ ".xlsx"))

In [ ]:
ic_dict = {
    'water consumption potential' : {'name': 'water use - water consumption potential (WCP)[cubic meter]',
                                                 'unit': 'm3-Eq',
                                                 'abbrev': 'WCP'},
    'freshwater ecotoxicity potential' : {'name': 'ecotoxicity: freshwater - freshwater ecotoxicity potential (FETP)[kg 1,4-DCB-Eq]',
                                                 'unit': 'kg 1,4-DCB-Eq',
                                                 'abbrev': 'FETP'},
    'particulate matter formation':  {'name': 'particulate matter formation - particulate matter formation potential (PMFP)[kg PM2.5-Eq]',
                                                 'unit': 'kg PM2.5-Eq',
                                                 'abbrev': 'PMFP'},
    'human toxicity potential, carcinogenics':  {'name': 'human toxicity: carcinogenic - human toxicity potential (HTPc)[kg 1,4-DCB-Eq]',
                                                 'unit': 'kg 1,4-DCB-Eq',
                                                 'abbrev': 'HTPc'},
    'surplus ore potential' : {'name': 'material resources: metals/minerals - surplus ore potential (SOP)[kg Cu-Eq]',
                                                 'unit': 'kg Cu-Eq',
                                                 'abbrev': 'SOP'},
    'ionising radiation potential' : {'name': 'ionising radiation - ionising radiation potential (IRP)[kg Co-60-Eq]',
                                                 'unit': 'kg Co-60-Eq',
                                                 'abbrev': 'IRP'}
    }

In [ ]:
def plot_ic (ic_name,ax): 

    # get the data
    name = ic_dict[ic_name]['name']
    df_ic_col_bioref_standalone = pd.DataFrame({'Stand-alone biorefinery': df_bioref_standalone_allICs[name]})
    df_ic_col_bioref_integrated = pd.DataFrame({'Integrated biorefinery': df_bioref_integrated_allICs[name]})
    df_ic_col_combustion = pd.DataFrame({'Combustion': df_combustion_allICs[name]})
    merged_df_table = pd.concat([df_ic_col_bioref_standalone, df_ic_col_bioref_integrated, df_ic_col_combustion], axis = 1)

    # get the total scores
    total_scores = list(merged_df_table.sum())
    total_scores = ['%.4f' % elem for elem in total_scores]
    total_scores = [float(elem) for elem in total_scores] # the list items must be floats

    # aggregate the EoL values into a unique row
    total_df = merged_df_table.copy()
    total_df.loc['EoL']= total_df.iloc[5:15].sum()
    total_df_vf = total_df.drop(total_df.index[5:15])

    # move the EoL row up
    row_to_move = total_df_vf.iloc[[10]]
    total_df_vf = total_df_vf.drop(['EoL'])
    total_df_vf = pd.concat([total_df_vf.iloc[:5], row_to_move, total_df_vf.iloc[5:]])
    
    #change index labels
    index_labels = ['lipophilic extraction', 'sugar extraction', 'pulping', 'heat (burd.)', 'heat (RB)','EoL', 
                    'tall oil (sub.)', 'starch (sub.)', 'phenol (sub.)', 'pulp (sub.)', 'heat CHP bark']
    
    total_df_vf.index = index_labels
    total_df_vf.rename(columns={'Stand-alone biorefinery': 'Sta.', 'Integrated biorefinery': 'Int.', 'Combustion':'Comb.'}, inplace=True)

    # plot the graph
    ic_abbrev = ic_dict[ic_name]['abbrev']
    ic_unit = ic_dict[ic_name]['unit']
    
    df_transposed = total_df_vf.T
    
    bars = df_transposed.plot(kind='bar', stacked=True, ax=ax, color=colors, legend=False)
    ax.set_ylabel(str(ic_abbrev + ' (' +ic_unit+')'), fontsize=10)
    # ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), title_fontsize='10', fontsize='10', ncol=4)
    
    ax.tick_params(axis='x', labelsize=10, rotation=0)
    ax.tick_params(axis='y', labelsize=10)
    ax.tick_params(axis='x', which='both', bottom=False)  # Remove x-axis ticks
    
    ax.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
    
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    
    # add the single GWP100 scores for each scenario
    categories_index = np.arange(len(new_names))
    ax.scatter(categories_index, total_scores, color='black', marker = 'H', zorder=5)

    import matplotlib.ticker as ticker
    formatter = ticker.ScalarFormatter(useMathText=True)
    formatter.set_powerlimits((-2, 2))
    ax.yaxis.set_major_formatter(formatter)
    ax.yaxis.get_offset_text().set_position((0.01, 1.02))
            
    
    handles, labels = plt.gca().get_legend_handles_labels()
    by_label = dict(zip(labels, handles))

    return by_label.values(), by_label.keys()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

fig = plt.figure(constrained_layout=True,figsize = (10,5))

ncols = 3
nrows = len(ic_dict.keys()) // ncols
keys = list(ic_dict.keys())
new_names = ['Stand-alone\nbiorefinery', 'Integrated\nbiorefinery', 'Combustion']


spec = gridspec.GridSpec(ncols=ncols, nrows=nrows, figure=fig,hspace=0.1,wspace=0.1)

handles = []
labels = []

count = 0
for i in range(ncols):
    for j in range(nrows): # n

        ax = fig.add_subplot(spec[j, i])      

     
        plot_ic (keys[count],ax = ax)

        plt.draw()
        labels, handles = ax.get_legend_handles_labels()

        if count>0:
            handles.append(handles)
            labels.append(labels)

        count+=1
leg = fig.legend(labels,handles, loc='lower center',bbox_to_anchor=(0.5, -0.15),ncols = 6,frameon = False)

plt.tight_layout()

plt.savefig(str(result_dir+'/SM1_lcia_ICs_' + ei + '.pdf'), dpi=300,bbox_extra_artists=[leg,], bbox_inches='tight')

plt.show()

In [ ]:
df_bioref_standalone_allICs.loc['Total'] = df_bioref_standalone_allICs.sum()
df_bioref_integrated_allICs.loc['Total'] = df_bioref_integrated_allICs.sum()
df_combustion_allICs.loc['Total'] = df_combustion_allICs.sum()

In [ ]:
new_df = pd.DataFrame(columns=list(df_bioref_standalone_allICs.columns))
new_df.loc['Stand-alone\nbiorefinery']=df_bioref_standalone_allICs.loc['Total']
new_df.loc['Integrated\nbiorefinery']=df_bioref_integrated_allICs.loc['Total']
new_df.loc['Combustion']=df_combustion_allICs.loc['Total']
df_transp = new_df.T
df_transp.to_excel(str(result_dir+"/table3_all_impacts_recipe_" + str(ei)+ ".xlsx"))
df_transp

# 8. Sensitivity analysis

### 8.1. Influence of the geographical location

The change of geographical location influences the indirect emissions from the biorefinery and combustion scneario only. In this model, the direct emissions remain similar, no matter the geographical location. The percentage of losses is fixed to 0.

In [ ]:
bioref_model = alg.newActivity(db_name = USER_DB,
                               name = 'bioref_model',
                               unit = 'unit',
                               exchanges={lipophilic_extraction:1, 
                                            sugar_extraction_withlosses:1, 
                                            pulping_withlosses:1, 
                                            heat_burdens:1, 
                                            #processA, 
                                            #processB, 
                                            #processC, 
                                            heat_residues:1, 
                                            # EoL
                                            EoL_pulp_biomed:1,
                                            EoL_pulp_textile:1, 
                                            EoL_pulp_nanocomposite:1,
                                            EoL_phenol_auto:1, 
                                            EoL_phenol_construction:1, 
                                            EoL_phenol_furniture:1, 
                                            EoL_phenol_electronics:1, 
                                            EoL_phenol_others:1, 
                                            EoL_starch_cardboard:1, 
                                            EoL_talloil_biofuel:1,                                   
                                            # substitution
                                            talloil_sub:1,
                                            starch_sub:1,
                                            phenol_sub:1, 
                                            pulp_sub:1})

In [ ]:
df_bioref_standalone_geoloc = alg.compute_impacts([bioref_model],
                                                  # lcia method
                                                  [ipcc2021],
                                                    # parameters
                                                    standalone = 1,
                                                    percentage_losses = 0,
                                                    grid_loc = ['grid_SE','grid_SE','grid_NO','grid_FI','grid_DK','grid_FR','grid_DE','grid_BE','grid_LU','grid_NL','grid_AT','grid_RER'],
                                                    mw_loc = ['mw_RER','mw_SE','mw_NO','mw_FI','mw_DK','mw_FR','mw_DE','mw_BE','mw_LU','mw_NL','mw_AT','mw_RER']
                                                  )

df_bioref_integrated_geoloc = alg.compute_impacts([bioref_model],
                                                  # lcia method
                                                  [ipcc2021],
                                                    # parameters
                                                    standalone = 0,
                                                    percentage_losses = 0,
                                                    grid_loc = ['grid_SE','grid_SE','grid_NO','grid_FI','grid_DK','grid_FR','grid_DE','grid_BE','grid_LU','grid_NL','grid_AT','grid_RER'],
                                                    mw_loc = ['mw_RER','mw_SE','mw_NO','mw_FI','mw_DK','mw_FR','mw_DE','mw_BE','mw_LU','mw_NL','mw_AT','mw_RER']
                                                  )

df_combustion_geoloc = alg.compute_impacts([heat_CHP_bark_scaled],
                            [ipcc2021],
                            # parameters
                            percentage_losses = 0,
                            grid_loc = ['grid_SE','grid_SE','grid_NO','grid_FI','grid_DK','grid_FR','grid_DE','grid_BE','grid_LU','grid_NL','grid_AT','grid_RER'],
                            mw_loc = ['mw_RER','mw_SE','mw_NO','mw_FI','mw_DK','mw_FR','mw_DE','mw_BE','mw_LU','mw_NL','mw_AT','mw_RER']
                            )

In [ ]:
# merge the three dataframes
merged_df_geoloc = pd.concat([df_bioref_standalone_geoloc, df_bioref_integrated_geoloc, df_combustion_geoloc], axis = 1)
# change the name of the columns
merged_df_geoloc.columns = ['Stand-alone biorefinery', 'Integrated biorefinery', 'Combustion']
merged_df_geoloc.index = ['Base.', 'SE', 'NO', 'FI', 'DK', 'FR', 'DE', 'BE', 'LU', 'NL', 'AT', 'RER']
# add the GWP100 for direct CO2 emissions (same for all scenarios) 
merged_df_geoloc+=1.83
merged_df_geoloc

In [ ]:
# Plotting the data
fig, ax = plt.subplots(figsize=(7, 5))

plt.scatter(merged_df_geoloc.index, merged_df_geoloc['Stand-alone biorefinery'], label='Stand-alone biorefinery', marker='o', color = '#332288', s=100)
plt.scatter(merged_df_geoloc.index, merged_df_geoloc['Integrated biorefinery'], label='Integrated biorefinery', marker='s', color = '#44AA99', s=100)
plt.scatter(merged_df_geoloc.index, merged_df_geoloc['Combustion'], label='Combustion', marker='^', color = '#CC6677', s=100)

# Adding titles and labels
plt.xlabel('Geographical location (country)', size =14)
plt.ylabel('Static climate impacts (kg CO2-eq/FU)', size=14)
plt.legend(frameon=False, fontsize=14)

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

ax.tick_params(axis='x', labelsize=14, rotation=0)
ax.tick_params(axis='y', labelsize=14)

    
plt.tight_layout()
plt.savefig(str(result_dir+'/SM1_sensitivity_geoloc_GWP100_'+ei+'.pdf'), dpi=300)


# Show plot
plt.show()

## 8.2. Influence of the background data

The choice of the background database influences the calculation of the indirect emissions. To perform this sensitivity analysis, the code above needs to be run until the end of Section 8.3, for each of the background databases. The results obtained are stored in Excel files that are imported below to plot the sensisitvity analysis graph. 

In [ ]:
# import the data from the Excel files
df_ei39 = pd.read_excel(str(result_dir+'/total_static_climate_impacts_IPCC_ecoinvent_3.9_conseq.xlsx'))
df_ei39_REMIND_SSP2_Base = pd.read_excel(str(result_dir+'/total_static_climate_impacts_IPCC_ecoinvent_consequential_3.9_remind_SSP2-Base_2045.xlsx'))
df_ei39_REMIND_SSP2_PkBudg500 = pd.read_excel(str(result_dir+'/total_static_climate_impacts_IPCC_ecoinvent_consequential_3.9_remind_SSP2-PkBudg500_2045.xlsx'))
df_ei39_REMIND_SSP1_PkBudg500 = pd.read_excel(str(result_dir+'/total_static_climate_impacts_IPCC_ecoinvent_consequential_3.9_remind_SSP1-PkBudg500_2045.xlsx'))
df_ei39_IMAGE_SSP2_Base = pd.read_excel(str(result_dir+'/total_static_climate_impacts_IPCC_ecoinvent_consequential_3.9_image_SSP2-Base_2045.xlsx'))

df_list = [df_ei39, df_ei39_REMIND_SSP2_PkBudg500, df_ei39_REMIND_SSP1_PkBudg500, df_ei39_REMIND_SSP2_Base, df_ei39_IMAGE_SSP2_Base]
db_names = ['Conventional', 'REMIND-SSP2-PkBudg500-2045', 'REMIND-SSP1-PkBudg500-2045', 'REMIND-SSP2-Base-2045','IMAGE-SSP2-Base-2045']

df_total_climate_impacts = pd.DataFrame(columns=['Stand-alone\nbiorefinery','Integrated\nbiorefinery','Combustion'])

for i in range(len(df_list)):
    df = df_list[i]
    # reset the index
    df = df.set_index('Unnamed: 0')
    # calculate the total value
    df.loc['Total'] = df.sum()
    # add the total into the new dataframe
    df_total_climate_impacts.loc[db_names[i]] = df.loc['Total'] 

df_total_climate_impacts

In [ ]:
# Marker styles and colors for different models
markers = ['o', 's', '^', 'D', 'p']
colors = ['#332288', '#44AA99', '#88CCEE', '#DDCC77', '#CC6677']
scenarios= ['Stand-alone\nbiorefinery','Integrated\nbiorefinery','Combustion']

# Create figure and axis
fig, ax = plt.subplots(figsize=(7,5))

# Set the positions of the x-ticks with reduced spacing
x_positions = (np.arange(len(scenarios))*4)  # Reduced spacing

# Plotting data points for each model in all scenarios
for j, db in enumerate(db_names):
    x_coords = []
    y_coords = []
    for i, scenario in enumerate(scenarios):
        x_coords.append(x_positions[i])  # Adjusted x-axis position
        y_coords.append(df_total_climate_impacts[scenario][db])  # Corresponding y-value for the model

    # Plot each model with a unique marker and color
    plt.scatter(x_coords, y_coords, marker=markers[j], facecolor = 'None',edgecolor=colors[j], s=100, label=db, linewidths=3)
    #plt.scatter(x_coords, y_coords, marker=markers[j], color=colors[j], s=100, label=model)

# Set labels and title#
plt.xticks(ticks=x_positions, labels=scenarios, size=14)
ax.set_xlim(-1.5,10)
#plt.xlabel("Scenarios")
plt.ylabel("Static climate impact (kg CO2-eq/FU)", size = 14)
#plt.title("Scatter Plot of Different Scenarios and Models")

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

ax.tick_params(axis='x', labelsize=14, rotation=0)
ax.tick_params(axis='y', labelsize=14)

# Add legend for models, with a smaller font size and adjusted position
plt.legend(frameon=False,bbox_to_anchor=(0.98, 0.98), loc='upper right', fontsize=14)

# Show grid
#plt.grid(True)

# Adjust layout to make room for the legend
plt.tight_layout()
plt.savefig(str(result_dir+'/SM1_sensitivity_background_GWP100.pdf'), dpi=300)

# Show the plot
plt.show()